In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
# autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
autolib_dir = '/'.join(nb_dir.split('/')[:-1])
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

/home/kai/data/shiyi/AlphaBoosting/automl/automl_app
['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [2]:
from automl_libs import base_layer_utils, stacknet
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs, utils 

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance'] 

In [4]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
#                                        fe.cumulative_count, 
#                                        fe.reverse_cumulative_count, 
#                                        fe.variance, fe.count_std_over_mean], 
#                                       2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
features_to_gen = get_features_to_gen([fe.count], 2, 2)#len(categorical_features))

features_to_gen 

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DepTime'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'UniqueCarrier'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Origin'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Dest'],
  'function': <function automl_libs.feature_engineering.co

In [5]:
len(features_to_gen)

28

In [6]:
project_path = './' 

In [7]:
from automl_app import logger_config
import logging
logger_config.config(project_path+'project.log', file_loglevel=logging.INFO)

In [8]:
def params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'svc':
        params = {
            'C': 0.911#np.random.rand(),
#             'metric': 'roc_auc'
        }
    elif model == 'logreg':
        params = {
            'penalty': np.random.choice(['l2']),
            'dual': np.random.choice([True, False]),
            'C': 1,#np.random.rand(),
            'metric': 'roc_auc'
        }
    elif model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 10, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc'
#             'objective': 'binary',
#             'boosting': 'gbdt',
#             'num_boost_round': 10, # ignored in params. extract it and put it in input arguments in train or cv explicitly
#                                    # seems to work fine as the upper limit when combined with early_stopping_round
#             'learning_rate': 0.01,
#             'num_leaves': 50,
#             'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
#             'max_depth': 5, # no limit. This is used to deal with over-fitting when #data is small.
#             'min_split_gain': 0.02,
#             'min_child_weight': 18.6,
#             'feature_fraction': 0.1,
#             'feature_fraction_seed': 1992,
#             'early_stopping_round':10,
#             'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
#             'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
#             'bagging_seed': 1992,
#             'scale_pos_weight': 1,
#             'lambda_l1': 0.8,
#             'lambda_l2': 0.6,
#             'metric': 'auc'
        }
    elif model == 'xgb':
        params = {
            'eta': 0.1,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'booster': 'gbtree',
            'tree_method': 'hist',
            'max_bin': 8,
            'scale_pos_weight': 1,
            'seed': seed,
            'nthread': 10,
            'max_depth': 6,
            'min_child_weight': 22,
            'gamma': 0,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'colsample_bylevel': 0.675,
            'alpha': 0,
            'lambda': 0,
            'num_boost_round': 50,
            'early_stopping_rounds': 1
        }
    elif model == 'catb':
        params = {
            'iterations': 30,
            'depth': np.random.randint(6,11),
            'l2_leaf_reg': np.random.randint(0, 31)/10,
        #     'custom_metric': 'AUC',
            'eval_metric': 'AUC',
            'random_seed': seed,
            'logging_level': 'Verbose',
            'thread_count': 15
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': np.random.randint(2,10),
            'lr_init': 0.01,
            'lr_fin': np.random.randint(1,5)/1000,  # if == lr_init, then no lr decay
            'batch_size': np.random.choice([128, 256, 512, 1024]),
            "pred_batch_size": 5000,
            'max_ep': 100,
            'patience': 10, #np.random.randint(10, 25),
            'cat_emb_outdim': 30, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
                                  # then assige embed_outdim_dict to cat_emb_outdim
            'cat_emb_drop_rate': np.random.randint(1,4)/10, 
            'num_layers_dense_units': [1000, 500, 100],
            'num_layers_drop_rate': np.random.randint(2,6)/10,
            'combined_layers_dense_units': [100, 50],
            'combined_layers_drop_rate': np.random.randint(1,3)/10,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    elif model == 'stacknet_layer2_nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.01, # if == lr_init, then no lr decay
            'batch_size': 128,
            "pred_batch_size": 50000,
            'best_epoch': 1,
            'patience': 1,
            'categorical_feature': [],
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'num_layers_dense_units': [],
            'combined_layers_dense_units': [10, 5],
            'combined_layers_drop_rate': 0,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    return params, seed

In [9]:
p, s = params_gen('stacknet_layer2_nn')
p, s

({'batch_size': 128,
  'best_epoch': 1,
  'cat_emb_outdim': 50,
  'categorical_feature': [],
  'combined_layers_dense_units': [10, 5],
  'combined_layers_drop_rate': 0,
  'ep_for_lr': 1,
  'int_list': ['num_layers_dense_units', 'combined_layers_dense_units'],
  'lr_fin': 0.01,
  'lr_init': 0.01,
  'mode': 'max',
  'monitor': 'val_auc',
  'nn_seed': 10723747,
  'num_layers_dense_units': [],
  'patience': 1,
  'pred_batch_size': 50000},
 10723739)

In [10]:
import sys
import time

class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
#         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('['+utils.get_time()+'] '+message)  
            self.log.close()


    def flush(self):
        #this flush method is needed for python 3 compatibility.
        #this handles the flush command by doing nothing.
        #you might want to specify some extra behavior here.
        pass    

def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

In [11]:
# divert_printout_to_file()

In [12]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, params_gen)

2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #22  | ==========START==========
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #67  | Run record file [./output/last_run_record.json] found. Continue from the previous run...
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #96  | generate todo list
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #112 | STAGE: FEATURE_ENGINEERING
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #116 | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #120 | STAGE: CONCAT_DATA
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting.__init__             | #124 | STAGE: GRID_SEARCH
2018-07-23 13:06:17 | INFO    | automl_app.app.AlphaBoosting._get_final_data      | #338 | DEBUG mode is on, 5.0% of train,val da

0:	learn: 0.6653179	test: 0.6672104	best: 0.6672104 (0)	total: 212ms	remaining: 6.15s
5:	learn: 0.6909695	test: 0.6868848	best: 0.6868848 (5)	total: 931ms	remaining: 3.73s
10:	learn: 0.6974981	test: 0.6945749	best: 0.6946138 (9)	total: 1.71s	remaining: 2.95s
15:	learn: 0.7006934	test: 0.6965080	best: 0.6965080 (15)	total: 2.52s	remaining: 2.21s
20:	learn: 0.7014892	test: 0.6971812	best: 0.6971874 (19)	total: 3.29s	remaining: 1.41s
25:	learn: 0.7028703	test: 0.6977995	best: 0.6977995 (25)	total: 4.12s	remaining: 634ms


2018-07-23 13:06:23 | INFO    | automl_libs.grid_search._catb_gs             | #410 | val_AUC: 0.69879 | train_AUC: 0.70354 (cv)
2018-07-23 13:06:23 | INFO    | automl_libs.grid_search._catb_gs             | #429 | [do_preds] is True, generating predictions ...
2018-07-23 13:06:23 | INFO    | automl_libs.grid_search._catb_gs             | #430 | Retrain model using best_round [29] and all data...


29:	learn: 0.7035405	test: 0.6987934	best: 0.6987934 (29)	total: 4.91s	remaining: 0us
0:	test: 0.6980526	best: 0.6980526 (0)	total: 66ms	remaining: 1.85s
1:	test: 0.7090833	best: 0.7090833 (1)	total: 134ms	remaining: 1.81s
2:	test: 0.7127950	best: 0.7127950 (2)	total: 193ms	remaining: 1.67s
3:	test: 0.7136455	best: 0.7136455 (3)	total: 261ms	remaining: 1.63s
4:	test: 0.7137311	best: 0.7137311 (4)	total: 321ms	remaining: 1.54s
5:	test: 0.7142073	best: 0.7142073 (5)	total: 384ms	remaining: 1.47s
6:	test: 0.7158434	best: 0.7158434 (6)	total: 453ms	remaining: 1.42s
7:	test: 0.7173511	best: 0.7173511 (7)	total: 520ms	remaining: 1.36s
8:	test: 0.7178568	best: 0.7178568 (8)	total: 585ms	remaining: 1.3s
9:	test: 0.7184095	best: 0.7184095 (9)	total: 652ms	remaining: 1.24s
10:	test: 0.7193606	best: 0.7193606 (10)	total: 715ms	remaining: 1.17s
11:	test: 0.7212022	best: 0.7212022 (11)	total: 790ms	remaining: 1.12s
12:	test: 0.7234478	best: 0.7234478 (12)	total: 858ms	remaining: 1.06s
13:	test: 0.7

2018-07-23 13:06:25 | INFO    | automl_libs.grid_search._catb_gs             | #437 | Training done. Iteration: 29 | train_AUC: 0.73764


27:	test: 0.7361731	best: 0.7361731 (27)	total: 1.94s	remaining: 69.3ms
28:	test: 0.7376363	best: 0.7376363 (28)	total: 2.01s	remaining: 0us

bestTest = 0.7376362785
bestIteration = 28

Shrink model to first 29 iterations.


2018-07-23 13:06:26 | INFO    | automl_libs.grid_search._catb_gs             | #442 | (_nn_gs) roc of test: 0.710457815308394
2018-07-23 13:06:26 | INFO    | automl_libs.grid_search._catb_gs             | #447 | CatB predictions(v5jC) saved in ./output/gs_saved_preds/.


> /home/kai/data/shiyi/AlphaBoosting/automl/automl_libs/grid_search.py(452)_catb_gs()
-> catb_params.update(res_dict)
(Pdb) catb_params
{'depth': 10, 'eval_metric': 'AUC', 'random_seed': 11164327, 'thread_count': 15, 'logging_level': 'Verbose', 'l2_leaf_reg': 2.4, 'iterations': 29}
(Pdb) res_dict
{'timestamp': '2018-07-23 13:06:17', 'cv': True, 'train_AUC': 0.7035405275421778, 'pred_timespent': '0:00:03', 'gs_timespent': '0:00:05', 'val_AUC': 0.6987933946090159}
(Pdb) n
> /home/kai/data/shiyi/AlphaBoosting/automl/automl_libs/grid_search.py(454)_catb_gs()
-> return catb_params, run_id
(Pdb) catb_params
{'timestamp': '2018-07-23 13:06:17', 'train_AUC': 0.7035405275421778, 'thread_count': 15, 'pred_timespent': '0:00:03', 'val_AUC': 0.6987933946090159, 'eval_metric': 'AUC', 'random_seed': 11164327, 'l2_leaf_reg': 2.4, 'iterations': 29, 'logging_level': 'Verbose', 'gs_timespent': '0:00:05', 'depth': 10, 'cv': True}
(Pdb) q


BdbQuit: 